# Zobecněný pseudopalindromický uzávěr

## Funkce pro vytvoření zobecněného pseudopalindromického uzávěru

In [1]:
import time
import itertools
import re

In [2]:
verbose = 0
verboseprint = lambda x,y: print(y) if verbose >= x else None

In [38]:
def timing(f):
    def wrap(*args):
        time1 = time.time()
        ret = f(*args)
        time2 = time.time()
        print('%s function took %0.3f ms' % (f.__name__, (time2-time1)*1000.0))
        return(ret)
    return wrap

In [4]:
def isPal(seq):
    "kontroluje jestli řetězec je palindrom"
    l = len(seq)
    if l == 1:
        return(True)
    for x in range(0, l//2):
        if seq[x] != seq[l-1-x]:
            return(False)
    return(True)

def isEpal(seq):
    "kontroluje jestli řetězec je pseudopalindrom"
    l = len(seq)
    if l%2 == 1:
        return(False)
    for x in range(0, l//2):
        if seq[x] == seq[l-1-x]:
            return(False)
    return(True)

In [5]:
def makePalClosure (seq):
    "udělá z řetězce palindromický uzávěr"
    if isPal(seq) == True:
        return(seq)
    i = 1
    while isPal(seq[i:]) != True:
        i = i+1
    verboseprint(2, "    {0} nejdelší palindromický uzávěr : {1}".format(seq,seq[i:]))
    verboseprint(2, "    délka nejdelší palindromický uzávěr : {0}".format(len(seq[i:])))
    closure = seq + seq[i-1::-1]
    return(closure)

def makeEpalClosure (seq):
    "udělá z řetězce pseudopalindromický uzávěr"
    if isEpal(seq) == True:
        return(seq)
    i = 1
    while isEpal(seq[i:]) != True:
        i = i+1
    verboseprint(2, "    {0} nejdelší pseudopalindromický uzávěr : {1}".format(seq,seq[i:]))
    closure = seq
    pref = seq[i-1::-1]
    for letter in pref:
        if letter == "0":
            closure = closure + "1"
        if letter == "1":
            closure = closure + "0"
    return(closure)

In [6]:
def makeWord(delta, theta, steps, seed = ""):
    "vytvoří slovo pomocí řídící posloupnosti a posloupnosti uzávěrů"
    w = seed
    for step in range(0,steps):
        w = w + delta[step]
        if theta[step] == "R":
            w = makePalClosure(w)
        if theta[step] == "E":
            w = makeEpalClosure(w)
        verboseprint(2, "w{0} = {1}".format(step+1,w))
    return(w)

In [7]:
def makeS(word):
    "udělá operaci S na slovo"
    Sword = ""
    for i in range(0,len(word)-1):
        Sword += str((int(word[i]) + int(word[i+1])) %2)
    return Sword

In [8]:
def isZps(word, closure = "ER", max_no_matters_closure_type = 0):
    '''kontroluje, jestli možné, aby slovo bylo získané zobec. pal. uzávěrem,
    pokud ano, vrací normalizovanou bidirektivní posloupnost''' 
    maximum = max_no_matters_closure_type
    l=1
    prefixes = []
    while l <= len(word):
        if ((l<=maximum or closure != "E") and isPal(word[:l])) or \
        ((l<=maximum or closure != "R") and isEpal(word[:l])):
            prefixes.append(word[:l])
        l=l+1
    verboseprint(1, prefixes)
    
    if not prefixes:
        verboseprint(1, "No prefixes of type " + str(closure) + " were found")
        return([False])
    if (len(prefixes[0]) > 2) or (len(prefixes[-1]) < len(word)//2) :
        return([False])        
        
    iszps = True
    i=0
    if closure != "E":
        newtheta = "R"
        newdelta = prefixes[0]
    else:
        newtheta = "E"
        newdelta = prefixes[0][0]
        
    while(i+1 < len(prefixes) and iszps == True):
        newletter = prefixes[i+1][len(prefixes[i])]
        condition_noE = (len(prefixes[i+1]) <= maximum) or (closure != "E")
        condition_noR = (len(prefixes[i+1]) <= maximum) or (closure != "R")
        if (condition_noE):
            palclo = makePalClosure(prefixes[i]+ newletter)
        if (condition_noR):
            epalclo = makeEpalClosure(prefixes[i]+ newletter)

        if(condition_noE and palclo == prefixes[i+1]):
            newtheta = newtheta + "R"
            newdelta = newdelta + newletter
        elif(condition_noR and epalclo == prefixes[i+1]):
            newtheta = newtheta + "E"
            newdelta = newdelta + newletter
        else:
            iszps = False
        i = i+1
    return([iszps, newdelta, newtheta])

In [44]:
def rindex(mylist, myvalue):
    return len(mylist) - mylist[::-1].index(myvalue) - 1
def isZps2(word, closure = "ER", max_no_matters_closure_type = 0):
    '''kontroluje, jestli možné, aby slovo bylo získané zobec. pal. uzávěrem,
    pokud ano, vrací normalizovanou bidirektivní posloupnost''' 
    maximum = max_no_matters_closure_type
    length = len(word)
    l=1
    prefixes = []
    lengths = []
    iszps = True
    newT = ""
    
    while l <= len(word) and iszps == True:
        
        while l <= len(word):
            if ((l<=maximum or closure != "E") and isPal(word[:l])):
                newT = "R"
                break
            elif ((l<=maximum or closure != "R") and isEpal(word[:l])):
                newT = "E"
                break
            l = l+1
        if l < length:
            prefixes.append([newT, word[l]])
            lengths.append(l)
        elif l == length:
            prefixes.append([newT, None])
            lengths.append(l)
        l = l + 1
        
        if not prefixes:
            verboseprint(1, "No prefixes of type " + str(closure) + " were found")
            return([False])
        if (len(prefixes[0][1]) > 2):
            return([False])
        
        if len(prefixes) < 2:
            pass
        else:
            new_wk = prefixes[-1]
            wk = prefixes[-2]
            if ([new_wk[0], wk[1] if new_wk[0]==wk[0] else str((int(wk[1])+1)%2)] in prefixes[:-2]):
                llps = lengths[rindex(prefixes[:-2],[new_wk[0], wk[1] if new_wk[0]==wk[0] else str((int(wk[1])+1)%2)])]
                if 2*lengths[-2] - llps != lengths[-1]:
                    iszps = False
                
    if (lengths[-1] < length//2): 
        return([False])
        
    
    newdelta = word[0]
    newtheta = ""
    llist = list(zip(*prefixes))
    newtheta = newtheta + ''.join(llist[0])
    newdelta = newdelta + ''.join(llist[1][:-1])
    return([iszps, newdelta, newtheta])

In [43]:
def subs(dic, n):
    s = dic["0"]
    i = 1
    while len(s) < n:
        s = s + dic[s[i]]
        i = i + 1
    return s[:n]


def timeiszps(words):
    return list(map(isZps, words))

def timeiszps2(words):
    return list(map(isZps2, words))

words = []
for rep1 in range(1,4):
    for rep2 in range(1,5):
        phi0 = ['0'+ ''.join(i) for i in itertools.product('01', repeat=rep1)]
        phi1 = [''.join(j) for j in itertools.product('01', repeat=rep2)]
        cart = [ k for k in itertools.product(phi0, phi1)]
        n = 100
        for rule in cart:
            dicti = {"0": rule[0], "1": rule[1]}
            words.append(subs(dicti, n))       
        
isZps2("000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000")

[True,
 '000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000',
 'RRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRR',
 [1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
  38,
  39,
  40,
  41,
  42,
  43,
  44,
  45,
  46,
  47,
  48,
  49,
  50,
  51,
  52,
  53,
  54,
  55,
  56,
  57,
  58,
  59,
  60,
  61,
  62,
  63,
  64,
  65,
  

In [11]:
def isZP(word):
    '''kontroluje, jestli možné, aby slovo bylo získané R-pal. uzávěrem,
    pokud ano, vrací normalizovanou bidirektivní posloupnost''' 
    l=1
    prefixes = []
    while l <= len(word):
        if isPal(word[:l]):
            prefixes.append(word[:l])
        l=l+1
    #print(prefixes)
    if not prefixes:
        #print("No prefixes of type " + str(closure) + " were found")
        return([False])
    if (len(prefixes[-1]) < len(word)//2) :
        return([False])
        
    iszps = True
    i=0
    newtheta= "R"
    newdelta= prefixes[0]
    while(i+1 < len(prefixes) and iszps == True):
        newletter = prefixes[i+1][len(prefixes[i])]
        palclo = makePalClosure(prefixes[i]+ newletter)

        if(palclo == prefixes[i+1]):
            newtheta = newtheta + "R"
            newdelta = newdelta + newletter
        else:
            iszps = False
        i = i+1
    return([iszps, newdelta, newtheta])

In [12]:
def isZE(word):
    '''kontroluje, jestli možné, aby slovo bylo získané E-uzávěrem,
    pokud ano, vrací normalizovanou bidirektivní posloupnost''' 
    l=1
    prefixes = []
    while l <= len(word):
        if isEpal(word[:l]):
            prefixes.append(word[:l])
        l=l+1
    #print(prefixes)
    if not prefixes:
        #print("No prefixes of type " + str(closure) + " were found")
        return([False])
    if (len(prefixes[-1]) < len(word)//2) :
        return([False])
    if (len(prefixes[0]) != 2):
        return([False])
        
    iszps = True
    i=0
    newtheta = "E"
    newdelta = prefixes[0][0]
    while(i+1 < len(prefixes) and iszps == True):
        newletter = prefixes[i+1][len(prefixes[i])]
        epalclo = makeEpalClosure(prefixes[i]+ newletter)

        if(epalclo == prefixes[i+1]):
            newtheta = newtheta + "E"
            newdelta = newdelta + newletter
        else:
            iszps = False
        i = i+1
    return([iszps, newdelta, newtheta])

In [13]:
bad_prefixes = ["(0R)*0E", "(1R)*1E", "(0R)+1E1E", "(1R)+0E0E"]
bad_factors = ["1R0E1E", "1R1E0E", "0R0E1E", "0R1E0E", "1E0R1R", "1E1R0R", "0E0R1R", "0E1R0R"]
def makeBiseq(delta, theta):
    """Makes one sequence from tje bi-sequence delta andm theta"""
    if len(delta) != len(theta):
            print("délky delta a theta nejsou stejné")
            return 
    s = ""
    for i in range(len(delta)):
        s = s + delta[i] + theta[i]
    return s

def isNormalized(delta, theta):
    biseq = makeBiseq(delta, theta)
    if biseq.startswith("0R1R") or biseq.startswith("1R0R"):
        print("je tam RR aa*")
        return False
    elif (re.match(bad_prefixes[0], biseq) != None) or (re.match(bad_prefixes[1], biseq) != None):
        print("je tam R^iE a^i")
        return False
    elif (re.match(bad_prefixes[2], biseq) != None) or (re.match(bad_prefixes[3], biseq) != None):
        print("je tam R^iEE a^ia*a*")
        return False
    elif any(x in biseq for x in bad_factors):
        print("ve slove je tt*t* abb*")
        return False
    else:
        return True

In [14]:
@timing
def testGPW_S_on_GPW(deltas, thetas, steps, seed = "", normalized = False, 
                      closure = "RE", max_no_matters_closure_type = 0):
    """Funkce, která všechny delty a thety otestuje, uděla prefixy, operaci S a pak vyzkouší,
    jestli získané slovo může být z zobec. pal. uz."""
    
    printed = False
    for delta in deltas:
        for theta in thetas:
            if(normalized == False or isNormalized(delta, theta)):
                word = makeWord(delta, theta, steps, seed)
                Sword = makeS(word)
                result = isZps(Sword, closure, max_no_matters_closure_type)
                if result[0]==True:
                    #print("Slovo u:" + word)
                    #print("S(u):" + Sword)
                    print("delta = {0}, theta = {1} : {2} ".format(delta, theta, result))
        print("")

In [15]:
# Počet kroků
steps = 4
# Všechny kombinace pro delty a thety
thetas = [''.join(i) for i in itertools.product('ER', repeat=steps)]
deltas = [''.join(i) for i in itertools.product('01', repeat=steps)]

In [16]:
word = makeWord("0111001010", "REERRRRRR", 8)
print(word)
result = isZps(word,"R", 1)
print(result)

01100110011001101100110011001100110110011001100110
[False, '0', 'R']


In [17]:
# Funkce, která všechny delty a thety otestuje, uděla prefixy, operaci S a pak vyzkouší,
# jestli získané slovo může být z zobec. pal. uz.
for delta in deltas:
        for theta in thetas:
            word = makeWord(delta, theta, steps)
            result = isZps(word)
            result2 = isZE(word)
            #print(delta, theta, result, result2)
            if result[0]== True:
                print(delta, theta, result, result2)

0000 EEEE [True, '01010101', 'RERERERE'] [True, '0000', 'EEEE']
0000 EEER [True, '0101010', 'RERERER'] [True, '000', 'EEE']
0000 EERE [True, '010100', 'RERERE'] [False, '00', 'EE']
0000 EERR [True, '010100', 'RERERR'] [False]
0000 EREE [True, '01000', 'REREE'] [False, '0', 'E']
0000 ERER [True, '01000', 'RERER'] [False, '0', 'E']
0000 ERRE [True, '01000', 'RERRE'] [False, '0', 'E']
0000 ERRR [True, '01000', 'RERRR'] [False]
0000 REEE [True, '0010101', 'RRERERE'] [False]
0000 REER [True, '001010', 'RRERER'] [False]
0000 RERE [True, '00100', 'RRERE'] [False]
0000 RERR [True, '00100', 'RRERR'] [False]
0000 RREE [True, '000101', 'RRRERE'] [False]
0000 RRER [True, '00010', 'RRRER'] [False]
0000 RRRE [True, '00001', 'RRRRE'] [False]
0000 RRRR [True, '0000', 'RRRR'] [False]
0001 EEEE [True, '0101011', 'REREREE'] [True, '0001', 'EEEE']
0001 EEER [True, '0101011', 'RERERER'] [True, '000', 'EEE']
0001 EERE [True, '010101', 'RERERE'] [True, '000', 'EEE']
0001 EERR [True, '0101010', 'RERERER'] [Tr

In [18]:
a = zip([1,2,3], [3,4,5])
print(list(a))

[(1, 3), (2, 4), (3, 5)]


In [19]:
a = [[2,3], [4,5]]
print([5,3] in a)

False


In [20]:
b = [3,4,5,6]
b[:-2]

[3, 4]